#### Figure 3
The result is from the following Python files: 

- src/wcsn_classify_test.py 
- src/LWT_wcsn_classify_test.py 
- src/UWA_wcsn_classify_test.py 

For the following 9 datasets: \
'Zhang_T', 'Kang_ctrl', 'Zheng68K', 'Baron_human', 'Muraro', 'Segerstolpe', 'AMB', 'TM', 'Baron_mouse' \
 Calculate the F1-score and accuracy for different cell types, which will be used to generate Figure 3A and Figure 3B.

##### Figure 3A

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['font.family'] = 'Times New Roman'

WCSGNet = [0.768, 0.865, 0.703, 0.978, 0.966, 0.993, 1.000, 0.927, 0.972]  
WCSGNet_LWT = [0.817, 0.869, 0.719, 0.978, 0.965, 0.994, 1.000, 0.925, 0.967]  
WCSGNet_UWA = [0.822, 0.873, 0.716, 0.980, 0.966, 0.995, 1.000, 0.932, 0.969]  

datasets = ["Zhang T", "Kang", "Zheng 68k", "Baron Human", "Muraro", "Segerstolpe", "AMB", "TM", "Baron Mouse"]

x = np.arange(len(datasets))  
width = 0.25  

plt.figure(figsize=(8, 4))

plt.bar(x - width, WCSGNet, width, label='WCSGNet', color='#72B6A1', edgecolor='white', linewidth=1)
plt.bar(x, WCSGNet_LWT, width, label='WCSGNet(LWT)', color='#E99675', edgecolor='white', linewidth=1)
plt.bar(x + width, WCSGNet_UWA, width, label='WCSGNet(UWA)', color='#95A3C3', edgecolor='white', linewidth=1)

plt.ylabel('Accuracy', fontsize=10, fontweight="bold")
plt.xticks(x, datasets, fontsize=10, rotation=45)  
plt.ylim(0.6, 1.0)  

ax = plt.gca()
ax.spines['top'].set_visible(False)  
ax.spines['right'].set_visible(False) 

plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.25), fontsize=10, ncol=3, frameon=False)

def add_value_labels(ax):
    for bar in ax.patches:
        height = bar.get_height()
        ax.annotate(f'{height:.3f}',  
                    xy=(bar.get_x() + bar.get_width() / 2, height),  
                    xytext=(0, 3),  
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=8,  
                    rotation=90)  

add_value_labels(ax)

plt.tight_layout()

plt.savefig('../Figures/version-1229/Figure_3A.png', format='png', dpi=1200, bbox_inches='tight')
plt.savefig('../Figures/version-1229/Figure_3A.svg', format='svg', dpi=1200, bbox_inches='tight')

plt.show()


##### Figure 3B

In [ ]:
# Accuracy

WCSGNet = [0.822, 	0.939, 	0.765, 	0.987, 	0.973, 	0.994, 	1.000, 	0.957, 	0.981]  
WCSGNet_LWT = [0.855, 	0.941, 	0.790, 	0.987, 	0.972, 	0.995, 	1.000, 	0.956, 	0.982]  
WCSGNet_UWA = [0.856, 	0.943, 	0.789, 	0.987, 	0.971, 	0.996, 	1.000, 	0.959, 	0.982 ]  


datasets = ["Zhang T", "Kang", "Zheng 68k", "Baron Human", "Muraro", "Segerstolpe", "AMB", "TM", "Baron Mouse"]


x = np.arange(len(datasets))  
width = 0.25  

plt.figure(figsize=(8, 4))

plt.bar(x - width, WCSGNet, width, label='WCSGNet', color='#72B6A1', edgecolor='white', linewidth=1)
plt.bar(x, WCSGNet_LWT, width, label='WCSGNet(LWT)', color='#E99675', edgecolor='white', linewidth=1)
plt.bar(x + width, WCSGNet_UWA, width, label='WCSGNet(UWA)', color='#95A3C3', edgecolor='white', linewidth=1)


plt.ylabel('Accuracy', fontsize=10, fontweight="bold")
plt.xticks(x, datasets, fontsize=10, rotation=45) 
plt.ylim(0.6, 1.0)  

ax = plt.gca()
ax.spines['top'].set_visible(False) 
ax.spines['right'].set_visible(False)  

plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.25), fontsize=10, ncol=3, frameon=False)

def add_value_labels(ax):
    for bar in ax.patches:
        height = bar.get_height()
        ax.annotate(f'{height:.3f}',  
                    xy=(bar.get_x() + bar.get_width() / 2, height), 
                    xytext=(0, 3),  
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=8,  
                    rotation=90)  

add_value_labels(ax)

plt.tight_layout()

plt.savefig('../Figures/version-1229/Figure_3B.png', format='png', dpi=1200, bbox_inches='tight')
plt.savefig('../Figures/version-1229/Figure_3B.svg', format='svg', dpi=1200, bbox_inches='tight')

plt.show()